In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import traceback


## 크롤링 확인

In [ ]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

# ✅ 리스트 페이지 접속
list_url = "https://linkareer.com/list/recruit?filterBy_activityTypeID=5&filterBy_categoryIDs=58&filterBy_status=OPEN&orderBy_direction=DESC&orderBy_field=RECENT&page=1"
driver.get(list_url)
time.sleep(2)  # 페이지 로딩 대기

results = []

# ✅ 리스트 안의 공고 개수
row_count = len(driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr'))

for i in range(1, row_count + 1):
    try:
        # 공고 클릭
        link_element = driver.find_element(By.XPATH, f'//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr[{i}]/td[2]/div/a/div/p')
        link_element.click()

        # 새 창 핸들 얻기
        driver.switch_to.window(driver.window_handles[-1])

        # ✅ 새 창에서 데이터 크롤링
        company_name_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/header/h2')))
        company_name = company_name_element.text.strip()

        company_type_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[1]/dd')))
        company_type = company_type_element.text.strip()

        position_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[5]/dd')))
        position = position_element.text.strip()

        qualification_texts = []
        idx = 13
        while True:
            try:
                para = driver.find_element(By.XPATH, f'//*[@id="DETAIL"]/section/div/p[{idx}]')
                text = para.text.strip()
                if text == '' or '다음' in text:
                    break
                qualification_texts.append(text)
                idx += 1
            except:
                break

        qualification = "\n".join(qualification_texts)

        # ✅ 데이터 저장
        results.append({
            '회사명': company_name,
            '기업형태': company_type,
            '모집직무': position,
            '자격요건': qualification
        })

        print(f"✅ {company_name} 크롤링 완료")

        # 새 창 닫기
        driver.close()

        # 원래 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])

        time.sleep(1)  # 다음 클릭 전에 살짝 대기

    except Exception as e:
        print(f"❌ {i}번째 공고 크롤링 중 오류: {e}")
        # 새 창이 열렸다면 닫고 돌아오기
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        continue

driver.quit()

# ✅ 결과 출력
for r in results:
    print(r)


✅ 플랜핏 크롤링 완료
✅ 리얼드로우 크롤링 완료
✅ 주식회사 화이트큐브 크롤링 완료
✅ 주식회사 화이트큐브 크롤링 완료
✅ 놀유니버스 크롤링 완료
✅ 진에어 크롤링 완료
✅ 이앤에스글로벌 크롤링 완료
✅ CJ푸드빌 크롤링 완료
✅ CJ푸드빌 크롤링 완료
✅ 삼성전자 크롤링 완료
✅ 코레일네트웍스 크롤링 완료
✅ 주식회사 놀유니버스 크롤링 완료
✅ 한미그룹 크롤링 완료
✅ 국가정보원 크롤링 완료
✅ NHN Cloud 크롤링 완료
✅ NHN Cloud 크롤링 완료
✅ NHN Cloud 크롤링 완료
✅ EY한영 크롤링 완료
✅ Toss 크롤링 완료
✅ SNOW 크롤링 완료
{'회사명': '플랜핏', '기업형태': '중소기업', '모집직무': 'IT/인터넷', '자격요건': '- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용자와 대화에 두려움이 없으며 사용자 관점에서 생각하며 사용자에게 관심이 많은 분\n- 데이터를 통해 문제를 파악하고 데이터를 측정하여 해당 데이터에 기반한 논리적인 의사 결정을 내릴 수 있는 분\n- 다양한 분야의 이해관계자(팀원 및 외부인)와 효율적인 커뮤니케이션이 가능한 분'}
{'회사명': '리얼드로우', '기업형태': '중소기업', '모집직무': '서울 마포구', '자격요건': '서류전형 - 면접전형 - 최종합격'}
{'회사명': '주식회사 화이트큐브', '기업형태': '스타트업', '모집직무': '서울 강남구', '자격요건': 'ex) 상품 소개서 개선을 통한 B2B 계약 전환율 상승, 광고 소재 효율 테스트 등'}
{'회사명': '주식회사 화이트큐브', '기업형태': '스타트업', '모집직무': '서울 강남구', '자격요건': '• 진짜 ‘성공’을 만들고자 하는 열망이 강한 분.\n• 이 업무를 왜 해야 하는지를 항상 고민하며, 해당 이유를 근거로 팀원들을 설득할 수 있는 분\n• 컴퓨터공학 전공 혹은 이에 준하는 지식 및 활용 역량이 있으신 분'}
{'회사명': '놀유니버스', '기업형태': '중견기

In [34]:
results

[{'회사명': '플랜핏',
  '기업형태': '중소기업',
  '모집직무': 'IT/인터넷',
  '자격요건': '- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용자와 대화에 두려움이 없으며 사용자 관점에서 생각하며 사용자에게 관심이 많은 분\n- 데이터를 통해 문제를 파악하고 데이터를 측정하여 해당 데이터에 기반한 논리적인 의사 결정을 내릴 수 있는 분\n- 다양한 분야의 이해관계자(팀원 및 외부인)와 효율적인 커뮤니케이션이 가능한 분'},
 {'회사명': '리얼드로우',
  '기업형태': '중소기업',
  '모집직무': '서울 마포구',
  '자격요건': '서류전형 - 면접전형 - 최종합격'},
 {'회사명': '주식회사 화이트큐브',
  '기업형태': '스타트업',
  '모집직무': '서울 강남구',
  '자격요건': 'ex) 상품 소개서 개선을 통한 B2B 계약 전환율 상승, 광고 소재 효율 테스트 등'},
 {'회사명': '주식회사 화이트큐브',
  '기업형태': '스타트업',
  '모집직무': '서울 강남구',
  '자격요건': '• 진짜 ‘성공’을 만들고자 하는 열망이 강한 분.\n• 이 업무를 왜 해야 하는지를 항상 고민하며, 해당 이유를 근거로 팀원들을 설득할 수 있는 분\n• 컴퓨터공학 전공 혹은 이에 준하는 지식 및 활용 역량이 있으신 분'},
 {'회사명': '놀유니버스',
  '기업형태': '중견기업',
  '모집직무': '경기 성남시 수정구',
  '자격요건': '기본적인 오피스 툴(특히 Excel 또는 Google 스프레드시트) 사용이 가능한 분\n팀과의 커뮤니케이션을 중요하게 생각하며, 협업을 긍정적으로 대하는 분'},
 {'회사명': '진에어', '기업형태': '대기업', '모집직무': '서울 강서구', '자격요건': ''},
 {'회사명': '이앤에스글로벌', '기업형태': '중견기업', '모집직무': '서울 강남구', '자격요건': ''},
 {'회사명': 'CJ푸드빌',
  '기업형태': 

In [3]:
# 테스트: 새 탭이 열리는지 확인

driver.get("https://linkareer.com/list/recruit?filterBy_activityTypeID=5&filterBy_categoryIDs=58&filterBy_status=OPEN&orderBy_direction=DESC&orderBy_field=RECENT&page=1")
time.sleep(3)

selector = '//table/tbody/tr[1]/td[2]//p'
element = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))

print("현재 윈도우:", driver.current_window_handle)
print("기존 핸들 목록:", driver.window_handles)

driver.execute_script("arguments[0].click();", element)
time.sleep(2)

print("클릭 후 윈도우:", driver.current_window_handle)
print("클릭 후 핸들 목록:", driver.window_handles)


현재 윈도우: 05258E12E69BD2B82AB180147D3F9CCC
기존 핸들 목록: ['05258E12E69BD2B82AB180147D3F9CCC', '763D6D694D25223D776C5FEF888F5E72']
클릭 후 윈도우: 05258E12E69BD2B82AB180147D3F9CCC
클릭 후 핸들 목록: ['05258E12E69BD2B82AB180147D3F9CCC', '763D6D694D25223D776C5FEF888F5E72', '046D32987200EF54434527999D735ECF']


In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://linkareer.com/list/recruit?filterBy_activityTypeID=5&filterBy_categoryIDs=58&filterBy_status=OPEN&orderBy_direction=DESC&orderBy_field=RECENT&page=1")
time.sleep(3)

try:
    selector = '//table/tbody/tr[1]/td[2]//p'
    element = wait.until(EC.presence_of_element_located((By.XPATH, selector)))
    print("공고 element 찾음:", element.text.strip())
except Exception as e:
    print("에러 발생:", e)


공고 element 찾음: [플랜핏] PO(신입/경력) 모집


In [32]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://linkareer.com/activity/239941")

# ✅ 회사명
company_name_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/header/h2')))
company_name = company_name_element.text.strip()

# ✅ 기업형태
company_type_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[1]/dd')))
company_type = company_type_element.text.strip()

# ✅ 모집직무
position_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[5]/dd')))
position = position_element.text.strip()

# ✅ 자격요건 제목 찾기
qualification_title_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="DETAIL"]/section/div/p[12]')))
# 자격요건 텍스트는 p[13]부터 시작 (구조에 맞게 조정)
qualification_texts = []
idx = 13
while True:
    try:
        para = driver.find_element(By.XPATH, f'//*[@id="DETAIL"]/section/div/p[{idx}]')
        text = para.text.strip()
        if text == '' or '다음' in text:  # 종료 조건 (필요시 수정)
            break
        qualification_texts.append(text)
        idx += 1
    except:
        break  # 더 이상 요소 없으면 종료

qualification = "\n".join(qualification_texts)

# ✅ 출력
print("회사명:", company_name)
print("기업형태:", company_type)
print("모집직무:", position)
print("자격요건:", qualification)

driver.quit()


회사명: 플랜핏
기업형태: 중소기업
모집직무: IT/인터넷
자격요건: - IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분
- 사용자와 대화에 두려움이 없으며 사용자 관점에서 생각하며 사용자에게 관심이 많은 분
- 데이터를 통해 문제를 파악하고 데이터를 측정하여 해당 데이터에 기반한 논리적인 의사 결정을 내릴 수 있는 분
- 다양한 분야의 이해관계자(팀원 및 외부인)와 효율적인 커뮤니케이션이 가능한 분


## 자격요건 위치 다름 문제를 해결

In [35]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

list_url = "https://linkareer.com/list/recruit?filterBy_activityTypeID=5&filterBy_categoryIDs=58&filterBy_status=OPEN&orderBy_direction=DESC&orderBy_field=RECENT&page=1"
driver.get(list_url)
time.sleep(2)

results = []
row_count = len(driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr'))

for i in range(1, row_count + 1):
    try:
        link_element = driver.find_element(By.XPATH, f'//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr[{i}]/td[2]/div/a/div/p')
        link_element.click()

        driver.switch_to.window(driver.window_handles[-1])

        # 기본 정보 크롤링
        company_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/header/h2'))).text.strip()
        company_type = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[1]/dd'))).text.strip()

        # 모든 p 태그 가져오기
        p_elements = driver.find_elements(By.XPATH, '//*[@id="DETAIL"]/section[1]/div/p')
        p_texts = [p.text.strip() for p in p_elements]

        # 자격요건 인덱스 찾기
        qual_idx = next((idx for idx, text in enumerate(p_texts) if '자격요건' in text or '자격 요건' in text), None)

        # 모집직무 인덱스 찾기
        pos_idx = next((idx for idx, text in enumerate(p_texts) if '모집 직무' in text or '세부 직무' in text), None)

        qualification_texts = []
        if qual_idx is not None:
            for t in p_texts[qual_idx + 1:]:
                if any(keyword in t for keyword in ['지원', '혜택', '우대', '다음', '근무']):
                    break
                qualification_texts.append(t)

        detail_position = ''
        if pos_idx is not None:
            detail_position = p_texts[pos_idx + 1] if pos_idx + 1 < len(p_texts) else ''

        qualification = "\n".join(qualification_texts)

        position_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[5]/dd')))
        position = position_element.text.strip()

        results.append({
            '회사명': company_name,
            '기업형태': company_type,
            '모집직무': position,
            '세부직무': detail_position,
            '자격요건': qualification
        })

        print(f"✅ {company_name} 완료")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        time.sleep(1)

    except Exception as e:
        print(f"❌ {i}번째 공고 오류: {e}")
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        continue

driver.quit()

for r in results:
    print(r)

✅ 플랜핏 완료
✅ 리얼드로우 완료
✅ 주식회사 화이트큐브 완료
✅ 주식회사 화이트큐브 완료
✅ 놀유니버스 완료
✅ 진에어 완료
✅ 이앤에스글로벌 완료
✅ CJ푸드빌 완료
✅ CJ푸드빌 완료
✅ 삼성전자 완료
✅ 코레일네트웍스 완료
✅ 주식회사 놀유니버스 완료
✅ 한미그룹 완료
✅ 국가정보원 완료
✅ NHN Cloud 완료
✅ NHN Cloud 완료
✅ NHN Cloud 완료
✅ EY한영 완료
✅ Toss 완료
✅ SNOW 완료
{'회사명': '플랜핏', '기업형태': '중소기업', '모집직무': 'IT/인터넷', '세부직무': '', '자격요건': '- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용자와 대화에 두려움이 없으며 사용자 관점에서 생각하며 사용자에게 관심이 많은 분\n- 데이터를 통해 문제를 파악하고 데이터를 측정하여 해당 데이터에 기반한 논리적인 의사 결정을 내릴 수 있는 분\n- 다양한 분야의 이해관계자(팀원 및 외부인)와 효율적인 커뮤니케이션이 가능한 분\n\n\n[인재 영입 프로세스]\n- 서류 전형 -> 1차 직무 인터뷰 -> 2차 컬쳐핏 인터뷰 -> 최종 합격\n\n'}
{'회사명': '리얼드로우', '기업형태': '중소기업', '모집직무': '서울 마포구', '세부직무': 'SW Engineer', '자격요건': '- 하단 상세 공고 참조\n\n[전형 안내]\n서류전형 - 면접전형 - 최종합격\n\n*이메일 접수: people@realdraw.ai\n(이력서, 포트폴리오)'}
{'회사명': '주식회사 화이트큐브', '기업형태': '스타트업', '모집직무': '서울 강남구', '세부직무': '콘텐츠 디자이너 인턴', '자격요건': '그래픽 툴(Figma, Photoshop, Illustrator 등)을 활용한 디자인이 가능하신 분\n비즈니스 성장을 위한 다양한 프로젝트 기획 및 디자인 가능하신 분\n디자인 결과물의 성과 데이터를 함께 확인하고 개선하여 ’비즈니스 성과’를 달성하고 싶은 분\nex) 상품 

In [36]:
results

[{'회사명': '플랜핏',
  '기업형태': '중소기업',
  '모집직무': 'IT/인터넷',
  '세부직무': '',
  '자격요건': '- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용자와 대화에 두려움이 없으며 사용자 관점에서 생각하며 사용자에게 관심이 많은 분\n- 데이터를 통해 문제를 파악하고 데이터를 측정하여 해당 데이터에 기반한 논리적인 의사 결정을 내릴 수 있는 분\n- 다양한 분야의 이해관계자(팀원 및 외부인)와 효율적인 커뮤니케이션이 가능한 분\n\n\n[인재 영입 프로세스]\n- 서류 전형 -> 1차 직무 인터뷰 -> 2차 컬쳐핏 인터뷰 -> 최종 합격\n\n'},
 {'회사명': '리얼드로우',
  '기업형태': '중소기업',
  '모집직무': '서울 마포구',
  '세부직무': 'SW Engineer',
  '자격요건': '- 하단 상세 공고 참조\n\n[전형 안내]\n서류전형 - 면접전형 - 최종합격\n\n*이메일 접수: people@realdraw.ai\n(이력서, 포트폴리오)'},
 {'회사명': '주식회사 화이트큐브',
  '기업형태': '스타트업',
  '모집직무': '서울 강남구',
  '세부직무': '콘텐츠 디자이너 인턴',
  '자격요건': '그래픽 툴(Figma, Photoshop, Illustrator 등)을 활용한 디자인이 가능하신 분\n비즈니스 성장을 위한 다양한 프로젝트 기획 및 디자인 가능하신 분\n디자인 결과물의 성과 데이터를 함께 확인하고 개선하여 ’비즈니스 성과’를 달성하고 싶은 분\nex) 상품 소개서 개선을 통한 B2B 계약 전환율 상승, 광고 소재 효율 테스트 등\n\n[전형 안내]\n서류전형 - 면접전형 - 최종합격'},
 {'회사명': '주식회사 화이트큐브',
  '기업형태': '스타트업',
  '모집직무': '서울 강남구',
  '세부직무': '백엔드 개발자 인턴',
  '자격요건': '• Python 또는 Node.js, Go 등 하나 이상의 언어에 익숙하신 

In [38]:
pd.DataFrame(results)

,회사명,기업형태,모집직무,세부직무,자격요건
0,플랜핏,중소기업,IT/인터넷,,- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용...
1,리얼드로우,중소기업,서울 마포구,SW Engineer,- 하단 상세 공고 참조\n\n[전형 안내]\n서류전형 - 면접전형 - 최종합격\n...
2,주식회사 화이트큐브,스타트업,서울 강남구,콘텐츠 디자이너 인턴,"그래픽 툴(Figma, Photoshop, Illustrator 등)을 활용한 디자..."
3,주식회사 화이트큐브,스타트업,서울 강남구,백엔드 개발자 인턴,"• Python 또는 Node.js, Go 등 하나 이상의 언어에 익숙하신 분\n•..."
4,놀유니버스,중견기업,경기 성남시 수정구,브랜드 마케팅 지원 담당자,"브랜드 콘텐츠와 마케팅에 관심이 많고, 즐겁게 배워보고 싶은 분\n디지털 콘텐츠 트..."
5,진에어,대기업,서울 강서구,,
6,이앤에스글로벌,중견기업,서울 강남구,,
7,CJ푸드빌,대기업,서울 중구,데이터 전략·분석 전문가,"- 4년제 학사 졸 이상\n- 관련 학과 : 컴퓨터공학, 산업공학, 경영정보, 문헌..."
8,CJ푸드빌,대기업,서울 중구,디지털서비스 APP 개발자,"- 4년제 대학 졸업자 이상\n- 컴퓨터공학, 산업공학, 소프트웨어공학 등 관련학과..."
9,삼성전자,대기업,경기 수원시 영통구,[VD사업부] Product Partnership,


## 여러 페이지 크롤링 기능 추가

In [40]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

results = []

# ✅ 크롤링할 페이지 범위 (1~5페이지 예시)
for page in range(1, 6):  # 필요시 1~원하는 페이지 범위 수정
    list_url = f"https://linkareer.com/list/recruit?filterBy_activityTypeID=5&filterBy_categoryIDs=58&filterBy_status=OPEN&orderBy_direction=DESC&orderBy_field=RECENT&page={page}"
    driver.get(list_url)
    time.sleep(2)

    print(f"📄 {page}페이지 접속 완료")

    # ✅ 메인 탭 핸들 저장
    main_window = driver.current_window_handle

    # 공고 row 개수 가져오기
    row_count = len(driver.find_elements(By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr'))
    print(f"🔍 {row_count}개의 공고 탐색 예정")

    for i in range(1, row_count + 1):
        try:
            link_element = driver.find_element(By.XPATH, f'//*[@id="__next"]/div[1]/div/main/div/section/div[2]/table/tbody/tr[{i}]/td[2]/div/a/div/p')
            link_element.click()

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # ✅ 기본 정보 크롤링
            company_name = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/header/h2'))).text.strip()
            company_type = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[1]/dd'))).text.strip()

            position_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[1]/div/main/div/div/section[1]/div/article/div/dl[5]/dd')))
            position = position_element.text.strip()

            # ✅ 모든 p 태그 텍스트 수집
            p_elements = driver.find_elements(By.XPATH, '//*[@id="DETAIL"]/section[1]/div/p')
            p_texts = [p.text.strip() for p in p_elements]

            # ✅ 자격요건/모집직무 인덱스 탐색
            qual_idx = next((idx for idx, text in enumerate(p_texts) if '자격요건' in text or '자격 요건' in text), None)
            pos_idx = next((idx for idx, text in enumerate(p_texts) if '모집 직무' in text or '세부 직무' in text), None)

            # ✅ 자격요건 텍스트 추출
            qualification_texts = []
            if qual_idx is not None:
                for t in p_texts[qual_idx + 1:]:
                    if any(keyword in t for keyword in ['지원', '혜택', '우대', '다음', '근무']):
                        break
                    qualification_texts.append(t)
            qualification = "\n".join(qualification_texts)

            # ✅ 세부직무 추출
            detail_position = ''
            if pos_idx is not None:
                detail_position = p_texts[pos_idx + 1] if pos_idx + 1 < len(p_texts) else ''

            # ✅ 결과 저장
            results.append({
                '회사명': company_name,
                '기업형태': company_type,
                '모집직무': position,
                '세부직무': detail_position,
                '자격요건': qualification
            })

            print(f"✅ {company_name} ({i}/{row_count}, page {page}) 크롤링 완료")

            # 새 탭 닫기 & 메인 탭으로 전환
            driver.close()
            driver.switch_to.window(main_window)

            time.sleep(1)

        except Exception as e:
            print(f"❌ {page}페이지 {i}번째 공고 오류: {e}")
            # 예외 시 새 탭 닫고 메인 탭 복귀
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(main_window)
            continue

driver.quit()

# ✅ DataFrame 변환 및 저장
df = pd.DataFrame(results)
df.to_csv('linkareer_crawling.csv', index=False, encoding='utf-8-sig')

print("🎉 크롤링 완료! CSV 저장됨.")

📄 1페이지 접속 완료
🔍 20개의 공고 탐색 예정
✅ 플랜핏 (1/20, page 1) 크롤링 완료
✅ 리얼드로우 (2/20, page 1) 크롤링 완료
✅ 주식회사 화이트큐브 (3/20, page 1) 크롤링 완료
✅ 주식회사 화이트큐브 (4/20, page 1) 크롤링 완료
✅ 놀유니버스 (5/20, page 1) 크롤링 완료
✅ 진에어 (6/20, page 1) 크롤링 완료
✅ 이앤에스글로벌 (7/20, page 1) 크롤링 완료
✅ CJ푸드빌 (8/20, page 1) 크롤링 완료
✅ CJ푸드빌 (9/20, page 1) 크롤링 완료
✅ 삼성전자 (10/20, page 1) 크롤링 완료
✅ 코레일네트웍스 (11/20, page 1) 크롤링 완료
✅ 주식회사 놀유니버스 (12/20, page 1) 크롤링 완료
✅ 한미그룹 (13/20, page 1) 크롤링 완료
✅ 국가정보원 (14/20, page 1) 크롤링 완료
✅ NHN Cloud (15/20, page 1) 크롤링 완료
✅ NHN Cloud (16/20, page 1) 크롤링 완료
✅ NHN Cloud (17/20, page 1) 크롤링 완료
✅ EY한영 (18/20, page 1) 크롤링 완료
✅ Toss (19/20, page 1) 크롤링 완료
✅ SNOW (20/20, page 1) 크롤링 완료
📄 2페이지 접속 완료
🔍 20개의 공고 탐색 예정
✅ 근로복지공단 (1/20, page 2) 크롤링 완료
✅ 새만금개발공사 (2/20, page 2) 크롤링 완료
✅ 해태제과식품 (3/20, page 2) 크롤링 완료
✅ NHN서비스 (4/20, page 2) 크롤링 완료
✅ 펄어비스 (5/20, page 2) 크롤링 완료
✅ 네이버웹툰 (6/20, page 2) 크롤링 완료
✅ 더존비즈온 (7/20, page 2) 크롤링 완료
✅ 주식회사 누리미디어 (8/20, page 2) 크롤링 완료
✅ MBC (9/20, page 2) 크롤링 완료
✅ CJ ENM (10/20, page 2) 크롤링 완료


In [41]:
pd.DataFrame(results)

,회사명,기업형태,모집직무,세부직무,자격요건
0,플랜핏,중소기업,IT/인터넷,,- IT 서비스 PM & PO 관련된 업무에 관해 실제로 경험이 있는 분\n- 사용...
1,리얼드로우,중소기업,서울 마포구,SW Engineer,- 하단 상세 공고 참조\n\n[전형 안내]\n서류전형 - 면접전형 - 최종합격\n...
2,주식회사 화이트큐브,스타트업,서울 강남구,콘텐츠 디자이너 인턴,"그래픽 툴(Figma, Photoshop, Illustrator 등)을 활용한 디자..."
3,주식회사 화이트큐브,스타트업,서울 강남구,백엔드 개발자 인턴,"• Python 또는 Node.js, Go 등 하나 이상의 언어에 익숙하신 분\n•..."
4,놀유니버스,중견기업,경기 성남시 수정구,브랜드 마케팅 지원 담당자,"브랜드 콘텐츠와 마케팅에 관심이 많고, 즐겁게 배워보고 싶은 분\n디지털 콘텐츠 트..."
...,...,...,...,...,...
95,한국하니웰,중견기업,서울 마포구,,
96,스타쉽엔터테인먼트,대기업,서울 강남구,,- 학력 : 초대졸 이상\n- 경력 : 무관(신입/경력)\n- 엔터테인먼트 및 문화...
97,경신,대기업,"인천 연수구, 인천 연수구, 경기 화성시, 전북특별자치도 군산시, 경기 의왕시",채용연계형 인턴-영업-원가,
98,앳홈,중소기업,"경영/사무, 마케팅/광고/홍보, IT/인터넷",인플루언서 커머스 AMD,"유튜브, 인스타그램 등 콘텐츠 플랫폼을 자주 보고, 트렌드에 관심 많은 분\n인플루..."
